# ICEBERG + TRINO 搭建笔记

# iceberg

## Dockerfile

``` yaml
version: "3"

services:
  spark-iceberg:
    image: tabulario/spark-iceberg
    container_name: spark-iceberg
    build: spark/
    networks:
      iceberg_net:
    depends_on:
      - rest
      - minio
    volumes:
      - ./warehouse:/home/iceberg/warehouse
      - ./notebooks:/home/iceberg/notebooks/notebooks
    environment:
      - AWS_ACCESS_KEY_ID=admin
      - AWS_SECRET_ACCESS_KEY=password
      - AWS_REGION=us-east-1
    ports:
      - 18888:8888
      - 18080:8080
      - 10000:10000
      - 10001:10001
  rest:
    image: apache/iceberg-rest-fixture
    container_name: iceberg-rest
    networks:
      iceberg_net:
    ports:
      - 18181:8181
    environment:
      - AWS_ACCESS_KEY_ID=admin
      - AWS_SECRET_ACCESS_KEY=password
      - AWS_REGION=us-east-1
      - CATALOG_WAREHOUSE=s3://warehouse/
      - CATALOG_IO__IMPL=org.apache.iceberg.aws.s3.S3FileIO
      - CATALOG_S3_ENDPOINT=http://minio:9000
  minio:
    image: minio/minio
    container_name: minio
    environment:
      - MINIO_ROOT_USER=admin
      - MINIO_ROOT_PASSWORD=password
      - MINIO_DOMAIN=minio
    networks:
      iceberg_net:
        aliases:
          - warehouse.minio
    ports:
      - 19001:9001
      - 19000:9000
    command: ["server", "/data", "--console-address", ":9001"]
  mc:
    depends_on:
      - minio
    image: minio/mc
    container_name: mc
    networks:
      iceberg_net:
    environment:
      - AWS_ACCESS_KEY_ID=admin
      - AWS_SECRET_ACCESS_KEY=password
      - AWS_REGION=us-east-1
    entrypoint: |
      /bin/sh -c "
      until (/usr/bin/mc config host add minio http://minio:9000 admin password) do echo '...waiting...' && sleep 1; done;
      /usr/bin/mc rm -r --force minio/warehouse;
      /usr/bin/mc mb minio/warehouse;
      /usr/bin/mc policy set public minio/warehouse;
      tail -f /dev/null
      "
networks:
  iceberg_net:
```

# Trino

## 配置

路径：/trino/etc/catalog

iceberg.properties
```
connector.name=iceberg
iceberg.catalog.type=rest
iceberg.rest-catalog.uri=http://192.168.2.18:18181
iceberg.rest-catalog.warehouse=s3://warehouse/
iceberg.file-format=PARQUET
fs.native-s3.enabled=true
s3.endpoint=http://192.168.2.18:19000
s3.region=us-east-1
s3.path-style-access=true
s3.aws-access-key=admin
s3.aws-secret-key=password
iceberg.compression-codec=ZSTD
```

es.properties
```
connector.name=elasticsearch
elasticsearch.host=192.168.2.4
elasticsearch.port=19200
elasticsearch.ignore-publish-address=true
```

mysql.properties
```
connector.name=mysql
connection-url=jdbc:mysql://192.168.2.4:13306
connection-user=s3cve_test
connection-password=x4YXhzRR4YW9MTPe
```

# trino 连接 iceberg

In [2]:
from trino.dbapi import connect

conn = connect(
    host="192.168.2.18",
    port=8180,
    user="user",
    catalog="iceberg",
    schema="nio_iceberg"
)

cur = conn.cursor()

In [24]:
cur.execute("SHOW SCHEMAS FROM iceberg").fetchall()

[['docs_example'], ['information_schema'], ['nio_iceberg'], ['nyc'], ['test']]

In [25]:
cur.execute("SHOW TABLES FROM nyc").fetchall()

[['taxis']]

In [27]:
import pandas as pd

def get_data(sql):
    results = cur.execute(sql).fetchmany(10)
    df = pd.DataFrame(results, columns=[desc[0] for desc in cur.description])
    return df

In [28]:
sql = "select * from nyc.taxis"
get_data(sql)

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.00,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.00,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.00,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.00,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.00,0.3,30.30,2.5,0.0
5,1,2022-01-01 00:40:15,2022-01-01 01:09:48,1.0,10.30,1.0,N,138,161,1,33.0,3.0,0.5,13.00,6.55,0.3,56.35,2.5,0.0
6,2,2022-01-01 00:20:50,2022-01-01 00:34:58,1.0,5.07,1.0,N,233,87,1,17.0,0.5,0.5,5.20,0.00,0.3,26.00,2.5,0.0
7,2,2022-01-01 00:13:04,2022-01-01 00:22:45,1.0,2.02,1.0,N,238,152,2,9.0,0.5,0.5,0.00,0.00,0.3,12.80,2.5,0.0
8,2,2022-01-01 00:30:02,2022-01-01 00:44:49,1.0,2.71,1.0,N,166,236,1,12.0,0.5,0.5,2.25,0.00,0.3,18.05,2.5,0.0
9,2,2022-01-01 00:48:52,2022-01-01 00:53:28,1.0,0.78,1.0,N,236,141,2,5.0,0.5,0.5,0.00,0.00,0.3,8.80,2.5,0.0


In [29]:
sql = "select sum(passenger_count) from nyc.taxis"
get_data(sql)

,_col0
0,21615218.0


In [30]:
sql = "SELECT passenger_count, COUNT(*) as count FROM nyc.taxis GROUP BY passenger_count"
get_data(sql)

,passenger_count,count
0,2.0,2307597
1,6.0,205207
2,5.0,313230
3,3.0,604392
4,8.0,55
5,4.0,259405
6,1.0,11350810
7,0.0,332457
8,NaN,512300
9,9.0,13
